In [2]:
import pandas as pd

In [3]:
merged_df = pd.read_csv('merged.csv')

**Investigate our match result:**
ISS项目很多是match到具体任务上的，尤其是业务部Tina的工作

还有很多看似完全一致的项目，因各种原因regex并没有match成功：
- GS230045-BJ07	华樾中心B座-BJ-朝阳区创远路28号院1号楼项目
- GS240072-BJ36 西安工商学院工程楼设计项目

**Decide to use NLP on both 具体任务 and 项目名称 columns**

In [4]:
merged_df['匹配'].unique()

array(['GS', 'No Match', 'ISS'], dtype=object)

In [6]:
import spacy

In [7]:
# Load Spacy model (use the medium or large model for better performance)
nlp = spacy.load('zh_core_web_sm') 

# Create function to compare similarity between two texts
def get_similarity(text1, text2):
    if pd.isna(text1) or pd.isna(text2):  # Skip if either text is NaN
        return 0
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    return doc1.similarity(doc2)

# Compare similarity for each row in the merged dataset
def find_best_match(merged_row, project_names):
    best_match = ''
    highest_similarity = 0
    if not pd.isna(merged_row):
        i = 0
        for project in project_names:
            i += 1
            similarity = get_similarity(merged_row, project)
            if similarity > highest_similarity:
                highest_similarity = similarity
                best_match = project
            print(merged_row, '<->', project, ', with ', similarity,  ' iterating project number ', i)

    return best_match, highest_similarity

# First, filter the rows where '匹配' == 'No Match'
# This is to increase efficiency
no_match_df = merged_df[merged_df['匹配'] == 'No Match']

In [ ]:
# Warning: take several minutes
tina = no_match_df[no_match_df['部门']=='业务部Tina']
merged_df.loc[tina.index, 'Desc_Name_Match_ISS'], merged_df.loc[tina.index, 'Desc_Name_ISS_Similarity'] = zip(*tina['具体任务'].apply(lambda task: find_best_match(task, iss['项目名称'])))

In [ ]:
# Warning: take several minutes
gs_missing_match = no_match_df[~pd.isna(no_match_df['项目编码']) & no_match_df['项目编码'].isin(gs['项目编码'])]
merged_df.loc[gs_missing_match.index, 'Name_Name_Match_GS'], merged_df.loc[gs_missing_match.index, 'Name_Name_GS_Similarity'] = zip(*gs_missing_match['项目名称'].apply(lambda task: find_best_match(task, gs['项目名称'])))

In [ ]:
# Warning: take several minutes
iss_missing_match = no_match_df[~pd.isna(no_match_df['项目编码']) & no_match_df['项目编码'].isin(iss['项目编码'])]
merged_df.loc[iss_missing_match.index, 'Name_Name_Match_ISS'], merged_df.loc[iss_missing_match.index, 'Name_Name_ISS_Similarity'] = zip(*iss_missing_match['项目名称'].apply(lambda task: find_best_match(task, iss['项目名称'])))

In [ ]:
merged_df[(merged_df['Name_Name_GS_Similarity'] != 0) & (~pd.isna(merged_df['Name_Name_GS_Similarity']))]['Name_Name_GS_Similarity'].unique()

In [ ]:
investigate = merged_df[(merged_df['Name_Name_GS_Similarity'] != 0) & (~pd.isna(merged_df['Name_Name_GS_Similarity']))]['Name_Name_GS_Similarity'].unique()[6]

In [ ]:
merged_df[merged_df['Name_Name_GS_Similarity'] == investigate]

**for project name matching: 0.85 is a threshold**

**Based on above matching result: match the ones above threshold, and output the full data for further manual investigation.**

In [ ]:
merged_df.to_csv('merged_with_similarity_results.csv', index=False, encoding='utf-8-sig')